To DO

+ ~~change landing page~~
+ ~~DONE add loading options to simulation; must explore useAsyncQueue NOPE (AsyncQueue too different from AsyncState)~~
+ ~~DONE add table; use icons for table segments; need to have column groups~~
+ ~~DONE!   need to add charts~~

+ ~~localstorage for chart values~~
+ ~~add env statement~~
+ ~~improve Chart aesthetics~~
    ~~+ window or tiles or windows with tiles~~
    ~~+ stack legend~~
    ~~+ use theme-related colors~~
    ~~+ pass appropriate x- and y-label values~~
+ ~~standardize container sizes and alignments
+ improve aesthetics of environment section
    + ~~improve chart layout~~
    + improve summary layout
+ ~~improve Table aesthetics~~
    + ~~add units for rows in Summary~~
+ check all math
+ ~~use Celery and Redis for async tasks to save monthly / quarterly / annual versions of statements~~
+ add temperature impacts / need to feed in variable chip temp
+ add current BTC environment modal
+ improve infra product database
+ improve infra section and remove errors
+ need full statements and charts for each project
+ need better atomicity of database transactions
+ push live to google
+ testing for drillbit
+ testing for drillbit-vj
+ testing for drillbit-dj
+ updates to finstat

Bugs / Questions / Clean-up
+ why is block schedule wrong?
+ number of miners???
+ Why do rot and istat not overlap in terms of number of periods??????
+ improve StatefulBtn functionality; how to use with useAsyncQueue????
+ aggregation and code clean-up???

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
import numpy as np
import pandas as pd

In [4]:
from drillbit import BitcoinEnvironmentUtility
from drillbit.__new_objects import Rig as RigManager, Product as ProductManager, \
    Cooling as CoolingManager, Project as ProjectManager, \
    OverClock, RigOperator
from drillbit.__new_units__ import AbstractBaseUnit, MagnitudeTable, Hashes, HashRate, Time, Power, Energy, Efficiency, EnergyPrice


from environment.serializers import EnvironmentSerializer as EnviroSer, BlockScheduleSerializer as BSchedSer, \
    BitcoinPriceSerializer as BPSer, TransactionFeesSerializer as FeeSer, HashRateSerializer as HRSer
from environment.models import Environment, BlockSchedule, BitcoinPrice, TransactionFees

from products.models import Rig, Cooling, HeatRejection, Electrical, RejectionCurve
from products.serializers import RigSerializer, CoolingSerializer, HeatRejectionSerializer, ElectricalSerializer

from projects.models import RigForProject, InfraForProject, Project, Projects, \
    ProjectStatement, ProjectSimulation, ProjectStatementSummary
from projects.serializers import RigForProjectSerializer, InfraForProjectSerializer, ProjectSerializer, ProjectsSerializer, \
    ProjectScalingSerializer, ProjectCostsSerializer, ProjectStatementSerializer, \
    ProjectStatementSummarySerializer

from projects.tasks import create_statements_for_given_project, create_statement_for_given_frequency

In [5]:
import numpy as np
import pandas as pd
import numpy_financial as npf

import finstat as fs

from drillbit.statements.funcs import total_energy, win_percentage, \
    expected_difficulty, hashes_to_hash_rate, win_percentage


In [6]:
from drillbit.statements.statements import init_environment, ProjectTemplate, ROITemplate, \
    analysis

In [7]:
btc = BitcoinEnvironmentUtility()
environment = Environment.objects.latest('created_at')
project = Project.objects.latest('created_at')
frequency = 'M'

In [8]:
projects = Projects.objects.latest('created_at').projects.all()

In [9]:
def custom_get_object(environment, project):
    sim = ProjectSimulation.objects.get(environment=environment, project=project)
    return ProjectStatementSummary.objects.get(sim=sim)


In [10]:
projects = Project.objects.filter(pk__in=projects)
summs = []
for project in projects:
    stat = custom_get_object(environment.id, project)
    ser = ProjectStatementSummarySerializer(stat)
    summs.append(ser.data['summary'])

df = pd.DataFrame(summs).T
df.columns = [p.name for p in projects]

In [12]:
df.reset_index().to_dict(orient='records')

[{'index': 'Capacity',
  'Project TEST #5': 40000000.0,
  'Project TEST #1': 40000000.0},
 {'index': 'Compute Power',
  'Project TEST #5': 37905709.54750059,
  'Project TEST #1': 37905709.54750059},
 {'index': 'Infra Power',
  'Project TEST #5': 2094290.452499412,
  'Project TEST #1': 2094290.452499412},
 {'index': 'Number of Rigs',
  'Project TEST #5': 11448.417259891448,
  'Project TEST #1': 9038.619266170033},
 {'index': 'Hash Rate',
  'Project TEST #5': 1.6027784163848028e+18,
  'Project TEST #1': 9.038619266170034e+17},
 {'index': 'Hash Rate per Rig',
  'Project TEST #5': 151108190137765.8,
  'Project TEST #1': 127154245861491.42},
 {'index': 'Total Hashes',
  'Project TEST #5': 1.3286013020806331e+26,
  'Project TEST #1': 8.826614910097902e+25},
 {'index': 'Energy Consumption',
  'Project TEST #5': 2792727272727273.0,
  'Project TEST #1': 2820196721311475.0},
 {'index': 'Efficiency',
  'Project TEST #5': 2.1020055214109535e-11,
  'Project TEST #1': 3.19510565492677e-11},
 {'index